# 缠论研究

In [2]:
import os

os.environ['czsc_research_cache']=os.environ.get('czsc_research_cache',os.getcwd()+'/data')

In [3]:
print(os.getenv('czsc_research_cache'))

/home/zzhang/dev/qi/github/portfolio/services/timescaledb/AShare/data


In [4]:
from czsc.connectors import research
from czsc import Event, Position

请设置tushare pro的token凭证码，如果没有请访问https://tushare.pro注册申请


ValueError: 请设置环境变量 czsc_research_cache 为投研共享数据的本地缓存路径，当前路径不存在：/home/zzhang/dev/qi/github/portfolio/services/timescaledb/AShare/data。

投研数据共享说明（含下载地址）：https://s0cqcxuy3p.feishu.cn/wiki/wikcnzuPawXtBB7Cj7mqlYZxpDh

本来要继续写上节所遗留的第一个中枢形成后走势类型的分类问题，但发现太多人，连每天如何看盘都搞不清楚，这事情可能更迫切，所以先说一下。

 

当然，如果是按某级别的严格操作，每天具体怎么走是关系不大的，走势不会因为交易是按天来的就有什么本质的不同。但针对每天的走势进行一些分类，至少是一个好的辅助。

 

一天的交易是4小时，等于有8个30分钟K线组成的一个系统。把3个相邻30分钟K线的重叠部分当成一个每天走势上的一个中枢，那么，一般来说显然，任何一天的走势，无非只有三类：一、只有一个中枢；二、两个中枢。三、没有中枢，其力度依次趋强。

 

一、只有一个中枢

 

这种走势，是典型的平衡市，一般情况下，开盘后前三根30分钟K线就决定了全天的波动区间，而全天的极限位置，基本上，至少有一个都出现在这前三根30分钟K线上，不是创出当天高点，就是创出当天低点。当然，这不是完全绝对的，因此可以对这种情况进行更细致的分类。

 

1、在前三根30分钟K线出现当天高点

 

这可以称为弱平衡市，其中最弱的是当天低点收，注意，这和当天是否红盘无关，高开最后红盘收也可以形成这种最弱的弱平衡市。次弱是收在中枢之下，收在中枢是一般的弱平衡市；收在当天高点附近的是最强的一种。

 

2、在前三根30分钟K线出现当天低点

 

这可以称为强平衡市，其中最强的，就是以当天高点收，同样，这与当天是否红盘无关。次强是收在中枢之上，收在中枢是一般的强平衡市；收在当天低点附近的是最弱的一种。

                       

3、在前三根30分钟K线不出现当天高低点

 

这可以称为转折平衡市，同样可以像上面情况一样根据收盘位置定义其强弱。

 

注意，以上三种情况中枢的位置不一定是前三根30分钟K线的重叠，可以是后面几根的。

 

 

二、两个中枢

 

显然，这根据两中枢的前后方向可以分为向上、向下两种，一般地，讨论向上的情况，向下的情况反过来就是。

 

两个中枢，显然不能有重叠的地方，否则就会转化成上面的情形。因此，这种形态，最大的特点就是这两个中枢之间有至少有一个30分钟K线，其中有部分区间是不属于两个中枢的任何一个，这个区间，成为单边区间，这是这种走势最重要的特点，是其后走势的关键位置。注意，具有单边区间的K线不从属任何一个中枢。

 

由于只有8根30分钟K线，根据单边区间所在位置，无非是两种可能： 1、单边区间在第四根30分钟K线。2、单边区间在第五根30分钟K线。由此就知道，为什么所有出现单边走势的，变盘时间都在中午收盘的前后30分钟之内。

 

当然，第4第5根30分钟K线可以同时具有单边区间。如果只有第4根K线具有单边区间的情况，那么第八根K线，有可能出现穿越单边区间的情况，例如，昨天200070417的走势。

 

显然，对于上涨的情况来说，最强的就是收盘在第二中枢的上方，最弱的，就是出现第八根K线出现穿越单边区间的情况，最终收在第一个中枢的之下。然后根据收盘的位置，可以依次定出其他的强弱。

 

三、没有中枢

 

这是最强的单边走势，8根K线，没有相临3根是有重叠部分的，一旦出现这种情况，就是典型的强烈走势，一旦出现这种走势，该日K线都是具有重要意义的。一般来说，这种走势很不常见。例如，227那天就是。但别以为出现这种走势就一定会继续趋势，往往很多骗线就是故意用这类走势构成，特别是在大的日K线中枢中出现这种情况，更大可能是骗线，例如227。当然，如果是在一个第三类买卖点后出现这种走势，出现大级别的强势趋势的可能性就极大了。

 

时间紧，不能写太多，如何利用每天走势图进行辅助操作，在周五的下节课中说

In [17]:
from datetime import datetime, timedelta
import tushare as ts
ts.set_token('7ac48d0a18070d8dc48e2027e88ed13b2e19c9d52bff35f9df1bd906')

def daily_classifier(ts_code, trade_date):
    """ A 股每日走势的分类

    使用该方法前，请仔细阅读：http://blog.sina.com.cn/s/blog_486e105c010009uy.html

    每个交易日产生 8 根 30 分钟 K 线，分别在前三根和最后三根中计算中枢，其结果分以下情况：
    1）没有中枢；
    2）仅有一个中枢，或计算得到的两个中枢区间有重叠；
    3）有两个中枢，且没有重叠。
    """
    start_date = datetime.strptime(trade_date, '%Y%m%d')
    end_date = start_date + timedelta(days=1)
    end_date = end_date.date().__str__().replace("-", "")

    df = ts.pro_bar(ts_code=ts_code, freq='30min', start_date=trade_date, end_date=end_date)
    df.sort_values('trade_time', inplace=True)
    data = df[['ts_code', 'trade_time', 'high', 'low']].iloc[1:, :]
    data = data.reset_index(drop=True)
    assert len(data) == 8, "每个交易日，A股有且只有8跟30分钟K线"

    def _central(tri):
        c_low = max(tri['low'])
        c_high = min(tri['high'])
        if c_low >= c_high:
            # None means no central found
            central = None
        else:
            central = {
                "time_span": "%s - %s" % (tri.iloc[0, 1], tri.iloc[2, 1]),
                "price_span": (c_low, c_high)
            }
        return central

    first_central = _central(data.iloc[:3, :])
    last_central = _central(data.iloc[-3:, :])
    return first_central, last_central

In [18]:
fc, lc = daily_classifier('600122.SH', "20190521")
print(fc, lc)

抱歉，您没有访问该接口的权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您没有访问该接口的权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您没有访问该接口的权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。


OSError: ERROR.